# DeepDream - PyTorch

We'll start by rewriting the deepdream code using PyTorch.
Once this will be done, we'll be able to play around with the training data in the network.

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms, utils
from torch.autograd import Variable

from PIL import Image
from PIL.JpegImagePlugin import JpegImageFile
from scipy.ndimage.filters import gaussian_filter


In [ ]:
torch.__version__

In [ ]:
if torch.cuda.is_available():
    print("GPU found")
    device = "cuda:0"
else:
    device = "cpu"

### DeepDream Algorithm

#### Helpers function

In [ ]:
def PIL_to_Tensor(image: Image):
    """
    Transform a PIL image into a Torch tensor.

    :param image: PIL image.
    :return: Tensor version of the PIL image.
    """
    loader = transforms.Compose([
        transforms.ToTensor()
    ])
    return loader(image)

def to_PIL(image: torch.Tensor):
    """
    Convert Tensor to PIL Image

    :param image: Tensor version of the PIL image.
    :return: PIL image.
    """
    return transforms.functional.to_pil_image(image.cpu())

def load_image(path: str):
    """
    Return a loaded image from a given path.

    :param path: (str) relative path of the iamge.
    :return: PIL image.
    """
    image = Image.open(path)
    return image

def save_image(image, filename: str):
    """
    Save image 'image' at path 'filename'

    :param image: Image to save.
    :param filename: (str) Path where to save the image.
    :return: None
    """
    # Convert Tensor into PIL Image if necessary
    if type(image) != Image.Image:
        tmp_image = to_PIL(image)
    else:
        tmp_image = image
    with open(filename, 'wb') as file:
        tmp_image.save(filename, 'jpeg')
    
def plot_image(image: torch.Tensor):
    """
    Plot an image

    :param image: Image to plot.
    :return: None
    """
    # Convert Tensor to PIL Image if necessary
    if type(image) == torch.Tensor:
        image = to_PIL(image)
    display(image)

def resize(inputs, size: tuple=None, factor: float=None, interpolation='bilinear', device='cuda:0'):
    """
    Resize  an image given a target size

    :param inputs: (Tensor, PIL.Image) Image
    :param size: (tuple<int, int>) Size of the new image.
    :param factor: (float) Scale factor by which multiply the dimension of the image.
    :param interpolation: (str) interpolation used
    :param device: (str) Device where to compute the model.
    :return: (Tensor, PIL.Image) resized image
    """
    # Get GPU if there is one
    if not torch.cuda.is_available() or 'cuda:' not in device:
        device = "cpu"

    is_image = False
    if type(inputs) != torch.Tensor:
        is_image = True
        inputs = PIL_to_Tensor(inputs).to(device).unsqueeze(0)

    with torch.no_grad():
        model = torch.nn.Upsample(
            size=size,
            scale_factor=factor,
            mode=interpolation,
            align_corners=True
        ).to(device)
        outputs = model(inputs)
    
    if is_image:
        outputs = to_PIL(outputs.squeeze())

    return outputs
    

In [ ]:
#######################
# DeepDream Algorithm #
#######################

class DeepDream(object):
    """docstring for DeepDream"""
    def __init__(self, image, network, device):
        super(DeepDream, self).__init__()
        self.image = image
        self.network = network
        self.device = device
        self.mean = torch.Tensor([0.485, 0.456, 0.406]).to(self.device)
        self.std = torch.Tensor([0.229, 0.224, 0.225]).to(self.device)

    def apply(self, at_layer, iterations: int, lr: float, octave_scale: float, num_octaves: int=None, blend: float=0.5, one_in: int=1):
        """
        Dream the image.

        Note: If only one of 'octave_scale' and 'num_octaves' are provided
              the other one will be automatically computed.
              IT IS NOT GUARANTEED TO WORK. 

        :param at_layer: (int, str) Layer to look at during dreaming.
        :param iterations: (int) Number of iterations of the Dream algorithms.
        :param lr: (float) Learning_rate of the Dream algorithm
        :param octave_scale: (float) Scale factor for the reduction of the image between two ocaves.
        :param num_octaves: (int) Number of octaves (Times we'll apply the scale factor)
        :param blend: (float) Blend factor between dreamed_image and normal image at each scale.
        :return: Dreamt image, transformed so that its nice.
        """
        #
        model = self.construct_model(self.network, at_layer)
        input_image = PIL_to_Tensor(self.image).to(self.device)
        
        # Complete parameters if not given.
        if num_octaves is None:
            min_dimension = min(input_image.size()[1:])
            num_octaves = math.floor(math.log(min_dimension / 32) / math.log(octave_scale))
        
        if octave_scale is None:
            max_dimension = max(input_image.size()[1:])
            octave_scale = math.pow(max_dimension / 224, 1.0 / num_octaves)

        # Transform the image in Tensor and preprocess it.
        input_image = to_PIL(input_image)
        preprocess = transforms.Compose([transforms.ToTensor(), transforms.Normalize(self.mean, self.std)])
        input_image = preprocess(input_image).unsqueeze(0).to(self.device)
        
        # Generate zoomed verion of the original image, and reapply to get more.
        octaves = [input_image]
        for _ in range(num_octaves // one_in - 1):
            new_octave = resize(octaves[-1], factor=1/octave_scale**one_in)
            octaves.append(new_octave)

        detail = torch.zeros_like(octaves[-1]).to(self.device)
        
        print("Octave: ", end="")
        for octave, octave_base in enumerate(octaves[::-1]):
            if octave > 0:
                # Upsample detail to new octave dimension
                detail = resize(detail, size=list(octave_base.size()[2:4]))

            # Add deep dream detail to new octave dimension
            input_image = 2.0 * (blend * octave_base + (1.0 - blend) * detail)

            # Get new deep dream image
            dreamed_image = self.dream(input_image, model, iterations, lr)
            
            # Extract deep dream details
            detail = dreamed_image - octave_base

            print(octave+1, end=" ")

        dreamed_image = to_PIL(self.deprocess(dreamed_image))
        print("Done !")
        return dreamed_image

    def dream(self, input: torch.Tensor, model, iterations: int, lr: float):
        """
        Main Dream function.
        Updates the image to maximize outputs for n iterations

        :param input: (Tensor) Image to dream on.
        :param model: Complete model to use for the dreaming.
        :param iterations: (int) Number of times we want to step toward the dream.
        :param lr: (float) Learning rate.
        :return: Dreamed image.
        """
        # Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
        input = Variable(input.to(self.device), requires_grad=True)
        for i in range(iterations):
            model.zero_grad()
            out = model(input)
            loss = out.norm()
            loss.backward()
            
            grad = input.grad.data.cpu().numpy()
            sigma = (i * 4.0) / iterations + 0.5
            grad_smooth1 = torch.Tensor(gaussian_filter(grad, sigma=sigma)).to(device)
            grad_smooth2 = torch.Tensor(gaussian_filter(grad, sigma=sigma*2)).to(device)
            grad_smooth3 = torch.Tensor(gaussian_filter(grad, sigma=sigma*0.5)).to(device)
            grad = grad_smooth1 + grad_smooth2 + grad_smooth3
            
            std_grad = grad.std()
            norm_lr = lr / std_grad
            input.data += norm_lr * grad.data
            input.data = self.clip(input.data)
            input.grad.data.zero_()
        return input

    # Helpers
    def clip(self, image_input: torch.Tensor):
        """
        Clip image according to mean and std in the network.

        :param image_input: (Tensor) Image.
        :return: Clipped image.
        """
        for c in range(3):
            m, s = self.mean[c], self.std[c]
            image_input[0, c] = torch.clamp(image_input[0, c], -m / s, (1 - m) / s)
        return image_input

    def deprocess(self, image_input: torch.Tensor):
        """
        Deprocess the image_input. Basically just unnormalize it.

        :param image_input: (Tensor) Image to unnormalize.
        :return: Deprocessed image.
        """
        image_input = image_input.squeeze() * self.std.reshape((3, 1, 1)) + self.mean.reshape((3, 1, 1))
        image_input = torch.clamp(image_input, min=0.0, max=1.0)
        return image_input

    def construct_model(self, network, at_layer):
        """
        Return a Sequential model ending by the 'at_layer'nth layer.

        :param network: (torchvision.models) Network used for the image recognition.
        :param at_layer: Layer where to stop the computation.
        :return: Model that return the output at layer 'at_layer' when computing on model 'network'
        """
        layers = list(network.features.children())
        model = nn.Sequential(*layers[: (at_layer + 1)])
        return model.to(self.device)


In [ ]:
image = load_image('images/dark_forest_2.jpg')
print("Image Loaded.")

In [ ]:
# Reduce size of image
image = resize(image, factor=3/4)

In [ ]:
# Get network
network = models.vgg19(pretrained=True)

In [ ]:
layers = range(23, 31)
dreamMachine = DeepDream(image, network, device)

for i in layers:
    print("Layer:", i)
    dreamed_image = dreamMachine.apply(
        at_layer=i,
        iterations=20,
        lr=0.015,
        octave_scale=1.428,
        num_octaves=4,
        blend=0.5,
    )
    plot_image(resize(dreamed_image, factor=4/3))
    #save_image(dreamed_image, f"images_dreamed/mush_{str(i)}.jpg")

In [ ]:
blend = range(0, 11)
dreamMachine = DeepDream(image, network, device)

for i in blend:
    print("Layer:", i)
    dreamed_image = dreamMachine.apply(
        at_layer=12,
        iterations=10,
        lr=0.01,
        octave_scale=1.428,
        num_octaves=4,
        blend=i/10,
    )
    plot_image(dreamed_image)
    #save_image(dreamed_image, f"images_dreamed/mush_{str(i)}.jpg")